# Salary Prediction
- [The Adult Salary Prediction dataset](https://archive.ics.uci.edu/ml/datasets/adult) consists of data from the 1994 US Census and the task is to predict whether a person earns `over $50K` a year (Class 1) or `less than $50K` a year (Class 0). The columns in the dataset are as follows:

|col name|description|
|:--|:--|
|age| continuous.|
|workclass| Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.|
|fnlwgt| continuous.|
|education| Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.|
|education-num| continuous.|
|marital-status| Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.|
|occupation| Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.|
|relationship| Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.|
|race| White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.|
|sex| Female, Male.|
|capital-gain| continuous.|
|capital-loss| continuous.|
|hours-per-week| continuous.|
|native-country| United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.|
|target| This is the target variable to be predicted. Class 1 for salary >50K and class 0 for salary <=50K|

- The goal of this project is to build and tune a model to predict the `target` column using AWS Sagemaker and deploy the model as a `Serverless Inference Endpoint`

## Tips: 
- You can use the below code to get the S3 bucket to write any artifacts to
    ```
    import sagemaker
    session = sagemaker.Session()
    bucket = session.default_bucket()
    ```
- Are all the columns necessary or we can drop any?
- What ML task is this? Classification? Regression? Clustering?
- How to determine the best hyperparameters for the model?
- How to test if the model is deployed successfully?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

cols = [
    "age", 
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "salary"
]

In [2]:
train_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", names=cols)
big_test_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", names=cols, skiprows=1)
train_df["salary"] = train_df["salary"].apply(lambda x: 1 if ">50K" in x else 0)
big_test_df["salary"] = big_test_df["salary"].apply(lambda x: 1 if ">50K" in x else 0)
train_df.to_csv('../data/train_data_salary_pred.csv',index=False);
big_test_df.to_csv('../data/test_data_salary_pred.csv',index=False);
print(train_df.shape, big_test_df.shape)
train_df.head()

(32561, 15) (16281, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


## All the best!
Get started below...

In [3]:
train_df.shape

(32561, 15)

In [4]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker-us-east-1-635439539142


In [5]:
big_test_df.to_json('../data/test_data_salary_pred_json.json',orient='records',lines=True,index=False)

In [6]:
train_path = session.upload_data(path='../data/train_data_salary_pred.csv', bucket=bucket, key_prefix = 'akash/salary-prediction')
test_path = session.upload_data(path='../data/test_data_salary_pred.csv', bucket=bucket, key_prefix = 'akash/salary-prediction')
print(f'train path: {train_path}')
print(f'test path: {test_path}')

train path: s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/train_data_salary_pred.csv
test path: s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/test_data_salary_pred.csv


In [7]:
train_df = pd.read_csv('s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/train_data_salary_pred.csv',nrows=1000)
train_df.shape

(1000, 15)

In [8]:
test_df = pd.read_csv('s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/test_data_salary_pred.csv',nrows=300)
test_df.shape

(300, 15)

In [9]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train_df,correlations={"auto": {"calculate": False}})
profile.to_file('profile_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# Separating the numerical and categorical columns

def data_type(dataset):
    """
    Function to identify the numerical and categorical data columns
    :param dataset: Dataframe
    :return: list of numerical and categorical columns
    """
    numerical = []
    categorical = []
    for i in dataset.columns:
        if dataset[i].dtype == 'int64' or dataset[i].dtype == 'float64':
            numerical.append(i)
        else:
            categorical.append(i)
    return numerical, categorical


numerical, categorical = data_type(train_df)

# Identifying the binary columns and ignoring them from scaling
def binary_columns(df):
    """
    Generates a list of binary columns in a dataframe.
    """
    binary_cols = []
    for col in df.select_dtypes(include=['int', 'float']).columns:
        unique_values = df[col].unique()
        if np.in1d(unique_values, [0, 1]).all():
            binary_cols.append(col)
    return binary_cols

binary_cols = binary_columns(train_df)

# Remove the binary columns from the numerical columns
numerical = [i for i in numerical if i not in binary_cols]

In [21]:
#split the data
X_train = train_df.drop('salary', axis=1)  # Features
y_train = train_df['salary']  # Target variable
X_test = test_df.drop('salary', axis=1)  # Features
y_test = test_df['salary']  # Target variable
print("Train data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Train data shape: (1000, 14) (1000,)
Test data shape: (300, 14) (300,)


In [22]:
from sklearn.preprocessing import StandardScaler
from category_encoders import CatBoostEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Create a pipeline for preprocessing
ct = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),  # StandardScaler for numeric features
        ('cat', CatBoostEncoder(), categorical)  # OneHotEncoder for categorical features
    ])

In [23]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


# Define classifiers
rfc = RandomForestClassifier()
gb = GradientBoostingClassifier()
logistic = LogisticRegression(penalty='l2',max_iter = 1000)
svm_rbf = SVC(kernel='rbf')

# Create pipelines for classifiers with the updated ColumnTransformer
rfc_pipeline = Pipeline([
    ("Data Transformations", ct),
    ("Random Forest", rfc)
])

gb_pipeline = Pipeline([
    ("Data Transformations", ct),
    ("Gradient Boosting", gb)
])

logistic_pipeline = Pipeline([
    ("Data Transformations", ct),
    ("Logistic Regression", logistic)
])

svm_rbf_pipeline = Pipeline([
    ("Data Transformations", ct),
    ("SVM with RBF kernel", svm_rbf)
])

# Fit and use the pipelines as before
rfc_pipeline.fit(X_train, y_train)
gb_pipeline.fit(X_train, y_train)
logistic_pipeline.fit(X_train, y_train)
svm_rbf_pipeline.fit(X_train, y_train)

# After fitting, you can use the pipelines for prediction and evaluation
rfc_predictions = rfc_pipeline.predict(X_test)
gb_predictions = gb_pipeline.predict(X_test)
logistic_predictions = logistic_pipeline.predict(X_test)
svm_rbf_predictions = svm_rbf_pipeline.predict(X_test)


In [24]:
from sklearn import set_config
set_config(display ="diagram")

In [25]:
gb_train_accuracy= gb_pipeline.score(X_train, y_train)
print(f"Gradient boost Training Accuracy:{gb_train_accuracy:.4f}")
gb_test_accuracy= gb_pipeline.score(X_test, y_test)
print(f"Gradient boost Test Accuracy:{gb_test_accuracy:.4f}")

Gradient boost Training Accuracy:0.8440
Gradient boost Test Accuracy:0.8533


In [26]:
rfc_train_accuracy= rfc_pipeline.score(X_train, y_train)
print(f"Random Forest Training Accuracy:{rfc_train_accuracy:.4f}")
rfc_test_accuracy= rfc_pipeline.score(X_test, y_test)
print(f"Random Forest Test Accuracy:{rfc_test_accuracy:.4f}")

Random Forest Training Accuracy:0.8730
Random Forest Test Accuracy:0.8533


In [27]:
logistic_train_accuracy= logistic_pipeline.score(X_train, y_train)
print(f"Logistic Ridge Training Accuracy:{logistic_train_accuracy:.4f}")
logistic_test_accuracy= logistic_pipeline.score(X_test, y_test)
print(f"Logistic Ridge Test Accuracy:{logistic_test_accuracy:.4f}")

Logistic Ridge Training Accuracy:0.8420
Logistic Ridge Test Accuracy:0.8467


In [28]:
svm_train_accuracy= svm_rbf_pipeline.score(X_train, y_train)
print(f"SVM Training Accuracy:{svm_train_accuracy:.4f}")
svm_test_accuracy= svm_rbf_pipeline.score(X_test, y_test)
print(f"SVM Test Accuracy:{svm_test_accuracy:.4f}")

SVM Training Accuracy:0.8630
SVM Test Accuracy:0.8333


In [29]:
%%writefile train.py
import argparse
import numpy as np
import os
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from category_encoders import CatBoostEncoder

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--rfc_n_estimators", type=int, default=100)
    parser.add_argument("--rfc_min_samples_split", type=float, default=0.05)
    parser.add_argument("--rfc_criterion", type=str, default="gini")
    parser.add_argument("--gb_n_estimators", type=int, default=100)
    parser.add_argument("--gb_learning_rate", type=float, default=0.1)
    parser.add_argument("--gb_max_depth", type=int, default=3)
    parser.add_argument("--logistic_max_iter", type=int, default=1000)
    parser.add_argument("--svm_kernel", type=str, default="rbf")
    args, _ = parser.parse_known_args()
    
    train_df = pd.read_csv('s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/train_data_salary_pred.csv',nrows=1000)  # Path to your train data file
    test_df = pd.read_csv('s3://sagemaker-us-east-1-635439539142/akash/salary-prediction/test_data_salary_pred.csv',nrows=300)   # Path to your test data file

    X_train = train_df.drop("salary", axis=1)
    y_train = train_df["salary"]
    X_test = test_df.drop("salary", axis=1)
    y_test = test_df["salary"] 

    def data_type(dataset):
        """
        Function to identify the numerical and categorical data columns
        :param dataset: Dataframe
        :return: list of numerical and categorical columns
        """
        numerical = []
        categorical = []
        for i in dataset.columns:
            if dataset[i].dtype == 'int64' or dataset[i].dtype == 'float64':
                numerical.append(i)
        else:
            categorical.append(i)
        return numerical, categorical


    numerical, categorical = data_type(X_train)

    # Identifying the binary columns and ignoring them from scaling
    def binary_columns(df):
        """
        Generates a list of binary columns in a dataframe.
        """
        binary_cols = []
        for col in df.select_dtypes(include=['int', 'float']).columns:
            unique_values = df[col].unique()
            if np.in1d(unique_values, [0, 1]).all():
                binary_cols.append(col)
        return binary_cols

    binary_cols = binary_columns(X_train)

    # Remove the binary columns from the numerical columns
    numerical = [i for i in numerical if i not in binary_cols]
    
    # Define your encoder
    ct = ColumnTransformer([
        ("CatBoostEncoding", CatBoostEncoder(), categorical),
        ("Scaling", StandardScaler(), numerical)
    ])
    
    # Define classifiers
    rfc = RandomForestClassifier(n_estimators=args.rfc_n_estimators, 
                                  min_samples_split=args.rfc_min_samples_split, 
                                  criterion=args.rfc_criterion)
    
    gb = GradientBoostingClassifier(n_estimators=args.gb_n_estimators, 
                                    learning_rate=args.gb_learning_rate, 
                                    max_depth=args.gb_max_depth)
    
    logistic = LogisticRegression(penalty='l2', max_iter=args.logistic_max_iter)
    
    svm_rbf = SVC(kernel=args.svm_kernel)
    
    # Create pipelines for classifiers with your encoder
    rfc_pipeline = Pipeline([
        ("Data Transformations", ct),
        ("Random Forest", rfc)
    ])

    gb_pipeline = Pipeline([
        ("Data Transformations", ct),
        ("Gradient Boosting", gb)
    ])

    logistic_pipeline = Pipeline([
        ("Data Transformations", ct),
        ("Logistic Regression", logistic)
    ])

    svm_rbf_pipeline = Pipeline([
        ("Data Transformations", ct),
        ("SVM with RBF kernel", svm_rbf)
    ])
    
    # Fit and evaluate each pipeline
    for pipeline, name in [(rfc_pipeline, 'Random Forest'), (gb_pipeline, 'Gradient Boosting'), 
                           (logistic_pipeline, 'Logistic Regression'), (svm_rbf_pipeline, 'SVM with RBF kernel')]:
        pipeline.fit(X_train, y_train)
        train_accuracy = pipeline.score(X_train, y_train)
        test_accuracy = pipeline.score(X_test, y_test)
        print(f"{name} Training Accuracy: {train_accuracy:.4f}")
        print(f"{name} Test Accuracy: {test_accuracy:.4f}")
        
        # Save the model
        model_save_path = os.path.join(args.model_dir, f"{name.lower().replace(' ', '_')}_model.joblib")
        joblib.dump(pipeline, model_save_path)
        print(f"Model Saved At: {model_save_path}")

if __name__ == "__main__":
    main()


Overwriting train.py


In [30]:
%%writefile requirements.txt
pandas
scikit-learn
fsspec
category_encoders
s3fs
botocore==1.27.18

Overwriting requirements.txt


In [31]:
!python train.py --model_dir ../models/ --rfc_n_estimators 100 --rfc_min_samples_split 0.05 --rfc_criterion gini --gb_n_estimators 100 --gb_learning_rate 0.1 --gb_max_depth 3 --logistic_max_iter 1000 --svm_kernel rbf

Random Forest Training Accuracy: 0.8510
Random Forest Test Accuracy: 0.8500
Model Saved At: ../models/random_forest_model.joblib
Gradient Boosting Training Accuracy: 0.8750
Gradient Boosting Test Accuracy: 0.8367
Model Saved At: ../models/gradient_boosting_model.joblib
Logistic Regression Training Accuracy: 0.8140
Logistic Regression Test Accuracy: 0.8200
Model Saved At: ../models/logistic_regression_model.joblib
SVM with RBF kernel Training Accuracy: 0.8370
SVM with RBF kernel Test Accuracy: 0.8300
Model Saved At: ../models/svm_with_rbf_kernel_model.joblib


In [33]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

sklearn_estimator = SKLearn(
    base_job_name="pipeline-run",
    framework_version="0.23-1",
    entry_point="train.py",
    dependencies=["requirements.txt"],  # Include the requirements file
    hyperparameters={
        "rfc_n_estimators": 100,
        "rfc_min_samples_split": 0.05,
        "rfc_criterion": "gini",
        "gb_n_estimators": 100,
        "gb_learning_rate": 0.1,
        "gb_max_depth": 3,
        "logistic_max_iter": 1000,
        "svm_kernel": "rbf"
    },
    instance_count=1,
    instance_type="ml.m5.large",
    use_spot_instances=True,
    max_wait=600,
    max_run=600,
    role=get_execution_role(),
)

sklearn_estimator.fit()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: pipeline-run-2024-03-03-08-34-54-446


Using provided s3_resource
2024-03-03 08:34:54 Starting - Starting the training job...
2024-03-03 08:35:10 Starting - Preparing the instances for training...
2024-03-03 08:35:49 Downloading - Downloading input data...
2024-03-03 08:36:12 Downloading - Downloading the training image...
2024-03-03 08:36:52 Training - Training image download completed. Training in progress...2024-03-03 08:36:57,925 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-03-03 08:36:57,929 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-03 08:36:57,977 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-03-03 08:36:58,169 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 23

In [35]:
import boto3
sm_client = boto3.client("sagemaker")
training_job_name = sklearn_estimator.latest_training_job.name
model_artifact = sm_client.describe_training_job(
    TrainingJobName = training_job_name
)["ModelArtifacts"]["S3ModelArtifacts"]

print(f"Training job name : {training_job_name}")
print(f"Model storage location : {model_artifact}")

Training job name : pipeline-run-2024-03-03-08-34-54-446
Model storage location : s3://sagemaker-us-east-1-635439539142/pipeline-run-2024-03-03-08-34-54-446/output/model.tar.gz


In [36]:
train_df.to_csv('../data/train_df.csv')
test_df.to_csv('../data/test_df.csv')

In [ ]:
import sagemaker
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

# Define the session, role, and S3 bucket
role = get_execution_role()

# Define hyperparameter ranges
hyperparameter_ranges = {
    "rfc_n_estimators": IntegerParameter(50, 150),
    "rfc_min_samples_split": ContinuousParameter(0.01, 0.5),
    "rfc_criterion": CategoricalParameter(["gini", "entropy"]),
    "gb_n_estimators": IntegerParameter(50, 150),
    "gb_learning_rate": ContinuousParameter(0.01, 0.5),
    "gb_max_depth": IntegerParameter(3, 10),
    "logistic_max_iter": IntegerParameter(100, 1000),
    "svm_kernel": CategoricalParameter(["linear", "poly", "rbf", "sigmoid"])
}

# Define the objective metric name and type
objective_metric_name = 'Training_Accuracy'
objective_type = 'Maximize'

# Define the metric definitions function
def generate_metric_definitions():
    return [
        {
            "Name": "Training_Accuracy",
            "Regex": "[a-zA-Z].*\\s+Training\\s+Accuracy:\\s+([0-9\\.]+)"
        },
        {
            "Name": "Test_Accuracy",
            "Regex": "[a-zA-Z].*\\s+Test\\s+Accuracy:\\s+([0-9\\.]+)"
        }
    ]


# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    base_tuning_job_name='pipeline-run',
    estimator=sklearn_estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    objective_type=objective_type,
    metric_definitions=generate_metric_definitions(),
    max_jobs=10,
    max_parallel_jobs=2
)

# Launch the hyperparameter tuning job
tuner.fit()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating hyperparameter tuning job with name: pipeline-run-240303-0855


..................................................................................................................................................................................

In [61]:
tuning_job_name = tuner.latest_tuning_job.name

(300, 15)

In [ ]:
best_hyperparameters = tuner.best_estimator().hyperparameters()

In [ ]:
tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuning_metrics = tuner_analytics.dataframe()

In [ ]:
import matplotlib.pyplot as plt

# Plot hyperparameter vs. objective metric
plt.scatter(tuning_metrics['parameter_<hyperparameter_name>'], tuning_metrics['FinalObjectiveValue'])
plt.xlabel('<Hyperparameter Name>')
plt.ylabel('Objective Metric Value')
plt.title('Hyperparameter Tuning Results')
plt.show()


In [ ]:
for job_name in tuner.analytics().training_job_summaries:
    print(f"Training Job Name: {job_name['TrainingJobName']}, Objective Metric Value: {job_name['FinalObjectiveValue']}")
